In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore


llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)


cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history",
)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the context. If you don't know the answer, just say you don't know; don't make it up.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"]


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("Is Aaronson guilty?")

Is Aaronson guilty?
content="According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, despite recalling that he had seen a photograph that disproved their guilt. However, he ultimately concludes that he had never seen the photograph and that it had never existed, indicating that he has accepted the Party's narrative."


In [3]:
invoke_chain("What message did he write in the table?")

What message did he write in the table?
content='Winston wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table. He also wrote "GOD IS POWER."'


In [4]:
invoke_chain("Who is Julia?")

Who is Julia?


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-dGme1RQQqZm5E5zjFKDX4tm4 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


content='Julia is a character who had a romantic relationship with Winston. She is portrayed as a rebellious figure against the Party, engaging in a love affair with Winston despite the oppressive regime they live under. Throughout the context, Winston expresses deep feelings for her and reflects on their connection, even after being tortured and broken by the Party.'


In [5]:
memory.load_memory_variables({})["history"]

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content="According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, despite recalling that he had seen a photograph that disproved their guilt. However, he ultimately concludes that he had never seen the photograph and that it had never existed, indicating that he has accepted the Party's narrative."),
 HumanMessage(content='What message did he write in the table?'),
 AIMessage(content='Winston wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table. He also wrote "GOD IS POWER."'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content='Julia is a character who had a romantic relationship with Winston. She is portrayed as a rebellious figure against the Party, engaging in a love affair with Winston despite the oppressive regime they live under. Throughout the context, Winston expresses deep feelings for her and reflects on their connec